## Data Import

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import datetime
from datetime import date
import matplotlib.pyplot as plt

In [2]:
# Read Data
stockx_data = pd.read_excel("../data/StockX-Data-Contest-2019.xlsx")
trend = pd.read_excel("../data/supplemental_data_google_trend.xlsx")
color = pd.read_excel("../data/supplemental_data_colorway.xlsx")

## Data Cleaning

In [3]:
# --- Data Cleaning ---
# calculate price premium: (sale price - retail price)/retail price
stockx_data["Pct_change"] = (stockx_data['Sale Price'] - stockx_data['Retail Price'])/stockx_data['Retail Price']
# calculate days since release: order data - release date
stockx_data["Days Since Release"] = (pd.to_datetime(stockx_data['Order Date']) - pd.to_datetime(stockx_data['Release Date']))/np.timedelta64('1','D')

# --- style variables ---
# style: v2
stockx_data["yeezy"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'Yeezy' in x.split("-") else 0)
# style: airjordan
stockx_data["airjordan"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'Jordan' in x.split("-") else 0)
# style: airforce
stockx_data["airforce"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'Force' in x.split("-") else 0)
# style: airmax90
stockx_data["airmax90"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if '90' in x.split("-") else 0)
# style: airmax97
stockx_data["airmax97"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if '97' in x.split("-") else 0)
# style: presto
stockx_data["presto"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'Presto' in x.split("-") else 0)
# style: vapormax
stockx_data["vapormax"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'VaporMax' in x.split("-") else 0)
# style: blazer
stockx_data["blazer"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'Blazer' in x.split("-") else 0)
# style: zoom
stockx_data["zoom"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'Zoom' in x.split("-") else 0)
# style: zoom
stockx_data["react"] = stockx_data['Sneaker Name'].apply(lambda x : 1 if 'React' in x.split("-") else 0)

# --- state variables ---
# creating the california variable
stockx_data["California"] = stockx_data["Buyer Region"].apply(lambda x : 1 if 'California' in x else 0)
# creating the new_york variable
stockx_data["New York"] = stockx_data["Buyer Region"].apply(lambda x : 1 if 'New York' in x else 0)
# creating the oregon variable
stockx_data["Oregon"] = stockx_data["Buyer Region"].apply(lambda x : 1 if 'Oregon' in x else 0)
# creating the florida variable
stockx_data["Florida"] = stockx_data["Buyer Region"].apply(lambda x : 1 if 'Florida' in x else 0)
# creating the texas variable
stockx_data["Texas"] = stockx_data["Buyer Region"].apply(lambda x : 1 if 'Texas' in x else 0)
# creating the other_state variable
above5pct_states = ["California", "New York", "Oregon", "Florida", "Texas"]
stockx_data["Other States"] = pd.Series(list(map(int,~stockx_data["Buyer Region"].isin(above5pct_states))))

# use frequency to encode shoe size
stockx_data['Shoe Size'] = stockx_data['Shoe Size'].astype(int)
frequency = stockx_data['Shoe Size'].value_counts(normalize=True).reset_index(name = 'size_freq')
stockx_data = stockx_data.merge(frequency, left_on = 'Shoe Size', right_on = 'index', how = 'left')
stockx_data = stockx_data.drop(columns = ['Shoe Size','index'])

## Data Merging

In [4]:
# merge colorway
stockx_data = stockx_data.merge(color, left_on = 'Sneaker Name', right_on = 'Style', how = 'left')

In [5]:
# merge google trend
#Step 1: get week number of google index on every sneaker style
trend['weeknum'] = trend['Week'].apply(lambda x : x.strftime("%V"))

#Step 2: get week number of stockx_data on every sale
stockx_data['weeknum'] = stockx_data["Order Date"].apply(lambda x : x.strftime("%V"))

#Step 3: Merge two tables by weeknum
#stockx_data = stockx_data.merge(trend, left_on = 'weeknum', right_on = 'weeknum', how = 'left')

#Step 4: Multiply style binary variables to get corresponding google trend index
#stockx_data['trend'] = (stockx_data['yeezy'].mul(stockx_data['Adidas Yeezy']) + stockx_data['airjordan'].mul(stockx_data['Air Jordan'])
#                        + stockx_data['airforce'].mul(stockx_data['Nike Air Force'])+
#                        stockx_data['airmax90'].mul(stockx_data['Nike Air Max']) +
#                        stockx_data['airmax97'].mul(stockx_data['Nike Air Max']) +
#                         stockx_data['presto'].mul(stockx_data['Nike Presto']) +
#                         stockx_data['vapormax'].mul(stockx_data['Nike Air VaporMax']) +
#                         stockx_data['blazer'].mul(stockx_data['Nike Blazers']) + 
#                         stockx_data['zoom'].mul(stockx_data['Nike Zoom']) + 
#                         stockx_data['react'].mul(stockx_data['Nike React']))

In [6]:
# drop columns that have been coded
# keep retail and sales prices for further analysis
# stockx_data = stockx_data.drop(columns = ['Retail Price','Order Date', 'Brand','Sneaker Name', 'Release Date', 'Buyer Region',
#                                          'Website', 'Product Line', 'Style','weeknum', 'Nike Air Force', 'Adidas Yeezy', 
#                                           'Nike Air Max', 'Nike Zoom','Nike Air VaporMax', 'Nike Blazers', 'Air Jordan', 
#                                           'Nike React','Nike Presto'])
stockx_data = stockx_data.drop(columns = ['Retail Price','Order Date', 'Brand','Sneaker Name', 'Release Date', 'Buyer Region',
                                         'Website', 'Product Line', 'Style','weeknum'])

In [7]:
# turn all the nan value to the 0
stockx_data = stockx_data.fillna(0)

## Save Data

In [8]:
# examine resulting dataframe
stockx_data.head()

,Sale Price,Pct_change,Days Since Release,yeezy,airjordan,airforce,airmax90,airmax97,presto,vapormax,...,Grey,Red,Green,Neo,Orange,Tan/Brown,Pink,Blue,Colorful,Number of Sales
0,1097.0,3.986364,342.0,1,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1857
1,685.0,2.113636,282.0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,560
2,690.0,2.136364,282.0,1,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,520
3,1075.0,3.886364,282.0,1,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,808
4,828.0,2.763636,202.0,1,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2153


In [9]:
# check dimension
stockx_data.shape

(99956, 32)

In [10]:
# save data to local
stockx_data.to_csv("../data/StockX-Data-Consolidated.csv")